# Purpose and Observation

## Purpose:
   - to read data from two datasets: 'CustSegList' and 'merged'
   - compare them to find relationships between them

## Observations:
  
### Main conclusion:
   - Thus, Merged dataset is wrapped up data of CustSegList on 'policy_owner_number'
   - Rather than a simple groupby, this is done by assessing individual columns differently
   - Categorical Variables:
        - these contain Identity Information which stays same across both datasets
    - Continuous Variables :
         - Any Identity Information stays same, eg. Owner salary
         - Variables related to Insurance policies change: 
             - sum_assured and afyp are added. 
             - PPT, policy term, billing frequency, max of two values is chosen
             - premium, smaller of the values is chosen for majority rows
    - DateTime:
         - DOB stays same
         - RCD, smaller of the values is chosen for majority rows
         - RCD for range for both datasets: 2013-01-01 to 2017-12-03
  
 ***Proceed on further analysis with 'merged' dataset***




# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv

# Read data and Basic Cleaning

In [ ]:
#import datasets
df_csl = pd.read_csv('CustSegList.csv')
df_merged = pd.read_csv('merged.csv')

#Marital_status, Own_Edu, Occupation_Group fix
df_merged.replace(['N', 'N.A', 'MISSING'], np.nan, inplace=True )
df_csl.replace(['N', 'N.A', 'MISSING'], np.nan, inplace=True )

df_csl.replace({'multi_cust': np.nan}, 'N', inplace = True)

#STATNAME fix and Focus_region fix

objectlistm = list(df_merged.select_dtypes('object').columns)
for col in objectlistm:
    df_merged[col] = df_merged[col].str.upper()

objectlistc = list(df_csl.select_dtypes('object').columns)
for col in objectlistc:
    df_csl[col] = df_csl[col].str.upper()

# putting them under SOUTH
df_csl['Focus_region'].replace(['KKG','ANDHRA','TAMIL NADU'], 'SOUTH', inplace = True)
df_merged['Focus_region'].replace(['KKG','ANDHRA','TAMIL NADU'], 'SOUTH', inplace = True)

# RCD and LA_DOB fix
df_merged["LA_DOB"]= pd.to_datetime(df_merged["LA_DOB"])
df_merged["RCD"] = pd.to_datetime(df_merged["RCD"])

df_csl["LA_DOB"]= pd.to_datetime(df_csl["LA_DOB"])
df_csl["RCD"] = pd.to_datetime(df_csl["RCD"])

# Descriptive Stats

In [ ]:
#observe data
display(df_merged.head())
display(df_csl.head())


#describe data
pd.options.display.float_format = "{:.2f}".format

display(df_merged.describe())
display(df_csl.describe())

# Find unique Identifier

In [ ]:
#checking differences
print("csl shape:",df_csl.shape, "\n",  
      "merged shape:", df_merged.shape, "\n",
      "rows difference between csl and merged:", abs(df_csl.shape[0]-df_merged.shape[0]), "\n"
      "column difference between csl and merged:", abs(df_csl.shape[1]-df_merged.shape[1]) )

In [ ]:
#checking ID number match
print("How many policy_owner_number entries of csl are present in merged: ",df_csl.loc[:,'policy_owner_number'].isin(df_merged.loc[:,'policy_owner_number']).sum(), "\n"
     "How many policy_number entries of csl are present in merged: ",df_csl.loc[:,'policy_number'].isin(df_merged.loc[:,'policy_number']).sum(), "\n"
     "How many Unnamed:0 entries of csl are present in merged : ",df_csl.loc[:,"Unnamed: 0"].isin(df_merged.loc[:,"Unnamed: 0"]).sum() )

#### Observations: 
- Total number of records for csl is 750598, same as number observed for entries of csl in merged.
- Confirmed with two candidates for identifiers: Policy Owner Number and Policy Number.
- Thus All records in CSL are present in Merged dataset.
- Unnamed: 0 can be dropped.
- Checking further now...

In [ ]:
#checkin duplicate values for ID
def check_duplicates(df, col):
    return abs(len(set(df.loc[:, col])) - len(df.loc[:, col]))

print("policy owner number".center(30,'-'),
      "csl: {}".format(check_duplicates(df_csl, "policy_owner_number")), 
      "merged: {}".format(check_duplicates(df_merged,"policy_owner_number")), sep="\n")

print("policy number".center(30,'-'),
      "csl: {}".format(check_duplicates(df_csl, "policy_number")), 
      "merged: {}".format(check_duplicates(df_merged,"policy_number")), sep="\n")

print("Unnamed: 0 :".center(30,'-'),
      "csl: {}".format(check_duplicates(df_csl, "Unnamed: 0")), 
      "merged: {}".format(check_duplicates(df_merged,"Unnamed: 0")), sep="\n")

#### Observation: 
- Duplicates only present for policy_owner_number in 'merged' dataset.
- The number of these duplicate values equals the difference between rows of merged and csl datasets
- Thus we can say that policy_number is unique identifier across both merged and csl
- And, policy_owner_number is unique identifier only for csl. Confirmed by the fact that the column, combined_policy contains multiple numbers which correspond to policy_number

In [ ]:
#drop Unnamed:0
x= "Unnamed: 0"
df_csl.drop([x], axis=1, inplace=True)
df_merged.drop([x], axis=1, inplace=True)

# Create a dataset df_diff which has rows present in merged, but not in csl

In [ ]:
#create list with uncommon values of policy number in two datasets
x= set(df_merged['policy_number'])-set(df_csl['policy_number'])
print(len(x)) #matches with the difference betn record numbers found earlier
l1= sorted(list(x))

#create new dataframe
df_diff= df_merged.loc[df_merged['policy_number'].isin(l1)]
df_diff.reset_index(inplace=True)
df_diff.drop(['index'], axis=1, inplace=True)
df_diff.head()

# Compare Merged and CSL datasets on common identifier( policy_number)

We compare every datapoint by
- selecting common policy_numbers in csl and merged
- selecting common columns in csl and merged
- comparing every datapoint by function get_disparities

In [ ]:
#common columns
com = sorted(list(set.intersection(set(df_csl.columns), set(df_merged.columns))))
print(com, "\n")

#uncommon columns in merged
unc_merged= sorted(list(set(df_merged.columns)-set.intersection(set(df_csl.columns), set(df_merged.columns))))
print(unc_merged, "\n")

#uncommon columns in csl
unc_csl= sorted(list(set(df_csl.columns)-set.intersection(set(df_csl.columns), set(df_merged.columns))))
print(unc_csl)

In [ ]:
def get_disparities(df1, df2):
    #common columns
    common = sorted(list(set.intersection(set(df1.columns), set(df2.columns))))
    
    #create dataframes
    global dfm1
    dfm1 = df1.loc[df1['policy_number'].isin(df2['policy_number']), common]
    global dfc1
    dfc1 = df2.loc[df2['policy_number'].isin(df1['policy_number']), common]
    
    #resetting index for new dataframe: essential for comparing
    dfm1.reset_index(inplace=True)
    dfm1.drop(['index'], axis=1, inplace=True)
    dfc1.reset_index(inplace=True)
    dfc1.drop(['index'], axis=1, inplace=True)
    
    #create new df for checking
    global df_check
    df_check = pd.DataFrame(columns=common)

    
    #Comparison
    #False indicates no mismatch
    for x in common:
        df_check[x]= np.where(dfm1[x] == dfc1[x], False, True)
    
    #value counts
    print("False indicates no mismatch")
    for col in df_check.columns: # can also call common instead of df_check.columns
        print(col.center(50,'*'), "\n", df_check[col].value_counts(), "\n")
    
    #make dictionary to locate disparities
    global location
    location= {}
    for i in df_check.index.values:
        for j in df_check:
            if df_check.loc[i,j] == True:
                if i in location.keys():
                    location[i].append(j)
                else:
                    location[i] = []
                    location[i].append(j)
    
    # Get variables with disparities
    l3= []
    for val in location.values():
        for i in val:
            l3.append(i)


    global dis_var 
    dis_var = list(set(l3))
    
    
    #getting index numbers with disparities
    global dis_ind 
    dis_ind = list(location.keys())
    
    print("variables with disparity:", len(dis_var))
    print("rows with disparity (unique from total disparities which tells about individual datapoints,i.e cells in csv, with mismatch", len(dis_ind))
    #uncomment below line of code to export newly created file
    
    """
    #create a merged dataframe to check them
    disparity = pd.merge(left=dfm1.loc[dis_ind, ['policy_number']+dis_var], right=dfc1.loc[dis_ind, dis_var], how="left", left_index=True, right_index=True, suffixes=('_merged', '_csl'))
    
    #create a location dataframe to find datapoint
    disparity_locations= pd.DataFrame.from_dict(location, orient="index")
    
    #export to excel
    with pd.ExcelWriter('Disparities.xlsx') as writer:
        disparity.to_excel(writer, sheet_name="Disparities")
        disparity_locations.to_excel(writer, sheet_name="Disparity_location")
    """


In [ ]:
get_disparities(df_merged, df_csl) #takes 4 minutes to run this function. modifications for efficiency needed
#False indicates no mismatch, True represent disparities

Observations :-
- 209752 disparities
- variables with disparities: 13
- records with disparities: 1,18,886
- max disparity on count: Own_Edu- 70084
- min disparity on count: city classification, DSTNAME, STATNAME- 11

## Checking for every column where disparity was found.
- greater than, equal to, less than relationships
- average change
- NOTE: NaN values cannot be compared, hence return false. check if there is actually disparity or simply missing values

## Checking categorical vars first

In [ ]:
#create dataframes to compare
for col in dis_var:
    globals()[col] = []

    for i in location.keys():
        if col in location[i]:
            globals()[col].append(i)
        
    globals()[col+'_df'] = pd.merge(left= dfm1.loc[globals()[col], col], right=dfc1.loc[globals()[col], col], left_index=True, right_index=True, suffixes=('_merged', '_csl'))
    
    
#Checking disparities
display(STATNAME_df)
display(DSTNAME_df)
display(City_classification_df)
display(Focus_region_df)
print("total null values in merged dataset for these indices", Focus_region_df['Focus_region_merged'].isna().sum())
print("total null values in csl dataset for these indices",Focus_region_df['Focus_region_csl'].isna().sum())

display(Marital_status_df)
print("total null values in merged dataset for these indices", Marital_status_df['Marital_status_merged'].isna().sum())
print("total null values in csl dataset for these indices",Marital_status_df['Marital_status_csl'].isna().sum())

display(Occupation_Group_df)
print("total null values in merged dataset for these indices", Occupation_Group_df['Occupation_Group_merged'].isna().sum())
print("total null values in csl dataset for these indices", Occupation_Group_df['Occupation_Group_csl'].isna().sum())

display(Own_Edu_df)
print("total null values in merged dataset for these indices", Own_Edu_df['Own_Edu_merged'].isna().sum())
print("total null values in csl dataset for these indices", Own_Edu_df['Own_Edu_csl'].isna().sum())

display(Own_gender_df)
print("total null values in merged dataset for these indices", Own_gender_df['Own_gender_merged'].isna().sum())
print("total null values in csl dataset for these indices", Own_gender_df['Own_gender_csl'].isna().sum())


**Thus we observe for above variables, we find disparity because these datapoints contains NaN values. NaN == NaN returns false hence they are counted as disparities. The above can thus be neglected**

## Checking continuous vars

- Condition:
    - if col1 > col2, categorize as 1
    - col1 < col2, categorize as 2
    - col1 == col2, categorize as 3
- Ideally we should have all as 3

In [ ]:
#afyp
conditions = [afyp_df.iloc[:, 0] > afyp_df.iloc[:, 1], afyp_df.iloc[:, 0] < afyp_df.iloc[:, 1], afyp_df.iloc[:, 0] == afyp_df.iloc[:, 1]]
choices = [1,2,3]
afyp_df['case'] = np.select(conditions, choices)
afyp_df['diff']=100*abs(afyp_df['afyp_merged']-afyp_df['afyp_csl'])/abs(afyp_df['afyp_merged'])
display(afyp_df)
print("value counts for defined cases", afyp_df['case'].value_counts())
display(afyp_df.mean())

#owner salary
display(Owner_salary_df)
print("total null values in merged dataset for these indices", Owner_salary_df['Owner_salary_merged'].isna().sum())
print("total null values in csl dataset for these indices", Owner_salary_df['Owner_salary_csl'].isna().sum())

#sum_assured
conditions = [sum_assured_df.iloc[:, 0] > sum_assured_df.iloc[:, 1], sum_assured_df.iloc[:, 0] < sum_assured_df.iloc[:, 1], sum_assured_df.iloc[:, 0] == sum_assured_df.iloc[:, 1]]
choices = [1,2,3]
sum_assured_df['case'] = np.select(conditions, choices)
sum_assured_df['diff']=100*abs(sum_assured_df['sum_assured_merged']-sum_assured_df['sum_assured_csl'])/abs(sum_assured_df['sum_assured_merged'])
display(sum_assured_df)
print("value counts for defined cases", sum_assured_df['case'].value_counts())
display(sum_assured_df.mean())
x= 100*abs(sum_assured_df['sum_assured_merged'].sum()-sum_assured_df['sum_assured_csl'].sum())/sum_assured_df['sum_assured_merged'].sum()
print("percent change on sum for sum assured (since 0 as value doesn't allow calculating percentage difference):", x)

#PPT
conditions = [PPT_df.iloc[:, 0] > PPT_df.iloc[:, 1], PPT_df.iloc[:, 0] < PPT_df.iloc[:, 1], PPT_df.iloc[:, 0] == PPT_df.iloc[:, 1]]
choices = [1,2,3]
PPT_df['case'] = np.select(conditions, choices)
PPT_df['diff']=100*abs(PPT_df['PPT_merged']-PPT_df['PPT_csl'])/abs(PPT_df['PPT_merged'])
display(PPT_df)
print("value counts for defined cases", PPT_df['case'].value_counts())
display(PPT_df.mean())

#Policy_term
conditions = [Policy_term_df.iloc[:, 0] > Policy_term_df.iloc[:, 1], Policy_term_df.iloc[:, 0] < Policy_term_df.iloc[:, 1], Policy_term_df.iloc[:, 0] == Policy_term_df.iloc[:, 1]]
choices = [1,2,3]
Policy_term_df['case'] = np.select(conditions, choices)
Policy_term_df['diff']=100*abs(Policy_term_df['Policy_term_merged']-Policy_term_df['Policy_term_csl'])/abs(Policy_term_df['Policy_term_merged'])
display(Policy_term_df)
print("value counts for defined cases", Policy_term_df['case'].value_counts())
print(Policy_term_df.mean())


## Observations of disparities:-

- All categorical variables disparity arises due to NaN i.e. missing values in both dataframe.
    - (reason: np.NaN == np.NaN returns False)
    
- For all continuous values except Owner_salary, CSL (CustSegList) has higher value than the corresponding datapoint in Merged
    - Owner_salary shows mismatch due to NaN values
    - afyp: mean percent increase in value is 194.51 %
    - sum assured: mean percent increase is Not Defined, due to presence of 0 as a value (which goes in denominator).
        - Percent change on sum of values is 131.5 %
    - PPT: mean percent increase in value is 92.5 %
    - Policy_term: mean percent increase in value is 92.64 %


# Hypothesis: CustSegList is consolidated on merged dataset, rather than subset of merged
- CustSegList is consolidated on Policy_owner_number

In [ ]:
#get all policy numbers from combined_policy as individual elements
l4 = []
x= list(df_csl['policy_number'])
for rec in df_csl['combined_policy']:
    rec = rec.split(',')
    for i in rec:
        l4.append(int(i))


In [ ]:
#check duplicates
print(abs(len(set(l4)) - len(l4)))

# check if all policy_nos of df_csl are present in combined policy
print("if all policy_nos of df_csl are present in combined policy", set(df_csl['policy_number']).issubset(set(l4)))

#getting policy nos not in df_csl
l5= list(set(l4).difference(set(df_csl['policy_number'])))

#check if all these are in df_diff and vice versa
print("if all policy nos not in df_csl are in df_diff", set(df_diff['policy_number']).issubset(set(l5)))
print("if in df_diff are policy nos not in df_csl", set(l5).issubset(set(df_diff['policy_number'])))

#comparing number of records
print(len(df_diff))
print(len(l5))

Thus all policy numbers in Merged are in [combined policy] in CustSegList  <br />
Also, the (number of these records) is same as (duplicate records for policy_owner_number in Merged) <br />
Thus concluding CustSegList has data wrapped up on policy_owner_number. <br />


## Checking continuous vars after grouping by policy_owner_number :
 - first by sum: afyp, sum_assured
 - then max: PPT, Policy_term, billing_frequency
 - then min: RCD, premium

#### 1. by sum

In [ ]:
#create dataframe grouped by policy owner number from Merged. 
# assuming afyp and sum assured are sum of combined policies

df_grp = df_merged.groupby(['policy_owner_number']).sum()
df_grp.head()

In [ ]:
#Compare
#afyp
rafyp_df = pd.merge(left= df_grp[['afyp']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','afyp']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))
rafyp_df['afyp_csl'] = rafyp_df['afyp_csl'].apply(np.ceil)
rafyp_df['afyp_mergrp'] = rafyp_df['afyp_mergrp'].apply(np.ceil)
rafyp_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rafyp_df)
conditions = [rafyp_df.iloc[:, 0] > rafyp_df.iloc[:, 1], 
              rafyp_df.iloc[:, 0] < rafyp_df.iloc[:, 1], 
              rafyp_df.iloc[:, 0] == rafyp_df.iloc[:, 1]]
choices = [1,2,3]
rafyp_df['case'] = np.select(conditions, choices)
rafyp_df['diff']=100*abs(rafyp_df['afyp_mergrp']-rafyp_df['afyp_csl'])/abs(rafyp_df['afyp_mergrp'])
print("value counts for cases:", rafyp_df['case'].value_counts())

#sum_assured
rsum_assured_df = pd.merge(left= df_grp[['sum_assured']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','sum_assured']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))
rsum_assured_df['sum_assured_csl'] = rsum_assured_df['sum_assured_csl'].apply(np.ceil)
rsum_assured_df['sum_assured_mergrp'] = rsum_assured_df['sum_assured_mergrp'].apply(np.ceil)
rsum_assured_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rsum_assured_df)
conditions = [rsum_assured_df.iloc[:, 0] > rsum_assured_df.iloc[:, 1], 
              rsum_assured_df.iloc[:, 0] < rsum_assured_df.iloc[:, 1], 
              rsum_assured_df.iloc[:, 0] == rsum_assured_df.iloc[:, 1]]
choices = [1,2,3]
rsum_assured_df['case'] = np.select(conditions, choices)
print("value counts for cases:", rsum_assured_df['case'].value_counts())



#### 2. by max

In [ ]:
#create dataframe grouped by policy owner number from Merged. 
# assuming PPT and Policy_term is taken as bigger of the two values in combined policy

y =['policy_owner_number',
 'policy_number',
 'PPT','Policy_term', 'billing_frequency']
df_grp = (df_merged.loc[:, y]).groupby(['policy_owner_number']).max()
df_grp.head()

In [ ]:
#PPT
rPPT_df = pd.merge(left= df_grp[['PPT']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','PPT']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))
rPPT_df['PPT_csl'] = rPPT_df['PPT_csl'].apply(np.ceil)
rPPT_df['PPT_mergrp'] = rPPT_df['PPT_mergrp'].apply(np.ceil)
rPPT_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rPPT_df)
conditions = [rPPT_df.iloc[:, 0] > rPPT_df.iloc[:, 1], rPPT_df.iloc[:, 0] < rPPT_df.iloc[:, 1], rPPT_df.iloc[:, 0] == rPPT_df.iloc[:, 1]]
choices = [1,2,3]
rPPT_df['case'] = np.select(conditions, choices)
print("value counts for cases:", rPPT_df['case'].value_counts())

#Policy_term
rPolicy_term_df = pd.merge(left= df_grp[['Policy_term']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','Policy_term']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))
rPolicy_term_df['Policy_term_csl'] = rPolicy_term_df['Policy_term_csl'].apply(np.ceil)
rPolicy_term_df['Policy_term_mergrp'] = rPolicy_term_df['Policy_term_mergrp'].apply(np.ceil)
rPolicy_term_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rPolicy_term_df)
conditions = [rPolicy_term_df.iloc[:, 0] > rPolicy_term_df.iloc[:, 1], rPolicy_term_df.iloc[:, 0] < rPolicy_term_df.iloc[:, 1], rPolicy_term_df.iloc[:, 0] == rPolicy_term_df.iloc[:, 1]]
choices = [1,2,3]
rPolicy_term_df['case'] = np.select(conditions, choices)
print("value counts for cases:", rPolicy_term_df['case'].value_counts())

#billing_frequency
rbilling_frequency_df = pd.merge(left= df_grp[['billing_frequency']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','billing_frequency']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))
rbilling_frequency_df['billing_frequency_csl'] = rbilling_frequency_df['billing_frequency_csl'].apply(np.ceil)
rbilling_frequency_df['billing_frequency_mergrp'] = rbilling_frequency_df['billing_frequency_mergrp'].apply(np.ceil)
rbilling_frequency_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rbilling_frequency_df)
conditions = [rbilling_frequency_df.iloc[:, 0] > rbilling_frequency_df.iloc[:, 1], rbilling_frequency_df.iloc[:, 0] < rbilling_frequency_df.iloc[:, 1], rbilling_frequency_df.iloc[:, 0] == rbilling_frequency_df.iloc[:, 1]]
choices = [1,2,3]
rbilling_frequency_df['case'] = np.select(conditions, choices)
print("value counts for cases:", rbilling_frequency_df['case'].value_counts())

#### 3. by min

In [ ]:
#create dataframe grouped by policy owner number from Merged. 
# assuming PPT and Policy_term is taken as bigger of the two values in combined policy

y =['policy_owner_number',
 'policy_number',
 'RCD', 'premium']
df_grp = (df_merged.loc[:, y]).groupby(['policy_owner_number']).min()
df_grp.head()

In [ ]:
#RCD
rRCD_df = pd.merge(left= df_grp[['RCD']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','RCD']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))

rRCD_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rRCD_df)
conditions = [rRCD_df.iloc[:, 0] > rRCD_df.iloc[:, 1], rRCD_df.iloc[:, 0] < rRCD_df.iloc[:, 1], rRCD_df.iloc[:, 0] == rRCD_df.iloc[:, 1]]
choices = [1,2,3]
rRCD_df['case'] = np.select(conditions, choices)
print("value counts for cases:", rRCD_df['case'].value_counts())

#premium
rpremium_df = pd.merge(left= df_grp[['premium']], 
             right=df_csl[df_csl['policy_owner_number'].isin(df_grp.index)][['policy_owner_number','premium']],
             left_index=True, 
             right_on= 'policy_owner_number', 
             suffixes=('_mergrp', '_csl'))
rpremium_df['premium_csl'] = rpremium_df['premium_csl'].apply(np.ceil)
rpremium_df['premium_mergrp'] = rpremium_df['premium_mergrp'].apply(np.ceil)
rpremium_df.drop(['policy_owner_number'], axis = 1, inplace = True)
display(rpremium_df)
conditions = [rpremium_df.iloc[:, 0] > rpremium_df.iloc[:, 1], rpremium_df.iloc[:, 0] < rpremium_df.iloc[:, 1], rpremium_df.iloc[:, 0] == rpremium_df.iloc[:, 1]]
choices = [1,2,3]
rpremium_df['case'] = np.select(conditions, choices)
print("value counts for cases:", rpremium_df['case'].value_counts())

# Main Observations:
    - Thus, Merged dataset is wrapped up data of CustSegList on 'policy_owner_number'
    - Rather than a simple groupby, this is done by assessing individual columns differently
    - Categorical Variables:
        - these contain Identity Information which stays same across both datasets
     - Continuous Variables :
         - Any Identity Information stays same, eg. Owner salary
         - Variables related to Insurance policies change: 
             - sum_assured and afyp are added. 
             - PPT, policy term, billing frequency, max of two values is chosen
             - premium, smaller of the values is chosen for majority rows
     - DateTime:
         - DOB stays same
         - RCD, smaller of the values is chosen for majority rows
         - RCD for range for both datasets: 2013-01-01 to 2017-12-03
  
      